![title](static/wcar.png)
# Interaction Partners for PIG-A
    Zhe Ji, Michele Tinti and Michael A.J. Ferguson

    Wellcome Centre for Anti-Infectives Research
    School of Life Sciences, University of Dundee

In [1]:
#reload library if modified
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, 'ProLib/')

In [3]:
#!ln -s /Users/mtinti/Desktop/git_projects/protUtil/ProLib ProLib

In [4]:
import os
from ProteomicsUtility import utilities as PTUT
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')

## Load Data

In [ ]:
#list(df.columns)

In [ ]:
KnownComponents={
'Tb11.v5.0765': 'TbPIGA',
'Tb927.5.3680': 'TbPIGH',
'Tb927.10.10110': 'TbPIGP',
'Tb927.10.6140': 'TbPIGC',
'Tb927.3.4570': 'TbPIGQ',
'Tb927.4.780': 'TbPIGY',
'Tb927.10.4610': 'TbDPM1',
'Tb927.9.6440': 'TbDPM2',
'Tb927.3.2480':'TbArv1'}

In [ ]:
#PTUT.make_vulcano??

In [ ]:
TXT_PATH = os.path.join('RUN_1', 'combined', 'txt')
#TXT_PATH=''
df = pd.read_csv(os.path.join(TXT_PATH, 'proteinGroups.txt'),sep='\t')
df = PTUT.clean_df(df, score=0, unique_pep_threshold=0)
df = PTUT.mod_df(df)

df['desc2'] = [ KnownComponents[g] if g in KnownComponents
               else d  for g,d in zip(df['Gene_id'],df['desc'])]

#tag = 'LFQ intensity'
#tag = 'iBAQ'
#tag='Intensity'
tag='MS/MS count'
bait_cols = [tag+' Bait_'+str(n) for n in range(1,4)]
print(df.shape)
#df = df[df[bait_cols].isna().sum(axis=1)==0]
print(df.shape)
control_cols = [tag+' Control_'+str(n) for n in range(1,4)]
df[bait_cols]=df[bait_cols].replace(0,np.nan)
df[control_cols]=df[control_cols].replace(0,np.nan)

#df = df[df[bait_cols].isna().sum(axis=1)==0] 

df[bait_cols]=df[bait_cols]/df[bait_cols].median()
df[control_cols]=df[control_cols]/df[control_cols].median()

df[bait_cols]=df[bait_cols].replace(np.nan,0)
df[control_cols]=df[control_cols].replace(np.nan,0)
for c in bait_cols+control_cols:
    df[c]=PTUT.impute(df[c])

 
    
df[bait_cols]=df[bait_cols]+df[bait_cols][df[bait_cols]>0].min()
df[control_cols]=df[control_cols]+df[control_cols][df[control_cols]>0].min()


df['FC']=np.log2(df[bait_cols].mean(axis=1,skipna=True)/
                 df[control_cols].mean(axis=1,skipna=True))
df['SumMSMS']=np.log10(df[bait_cols].mean(axis=1,skipna=True)+
                 df[control_cols].mean(axis=1,skipna=True))

fig, ax=plt.subplots(figsize=(12,8), ncols=1, nrows=1)
ids = df[df.Gene_id.isin(KnownComponents.keys())]
_ids = ids.index.values
_names = ids.desc2
print(len(_ids))
print(len(_names))
PTUT.make_vulcano(df, ax, 
                       x='SumMSMS', y='FC', 
             annot_index=_ids,
             annot_names = _names,
             title='MA',
              fc_col = 'log2FC',
                fc_limit=False,
            point_size_selection=20,
    point_size_all=10,
                  text_size=14,
                  alpha_main=0.1
                  
            )



In [ ]:
TXT_PATH = os.path.join('RUN_1', 'combined', 'txt')
#TXT_PATH=''
df = pd.read_csv(os.path.join(TXT_PATH, 'proteinGroups.txt'),sep='\t')
df = PTUT.clean_df(df, score=4, unique_pep_threshold=2)
df = PTUT.mod_df(df)
print(df.shape)
df.head()

In [ ]:
ml_df=pd.read_csv('final_df.csv.gz',index_col=[0])
aboundance_df = ml_df[['Gene ID','Abundance_BSF_pro']]
aboundance_df.set_index('Gene ID',inplace=True)
aboundance_df['BSF_rank']=aboundance_df['Abundance_BSF_pro'].rank(ascending=False)
aboundance_df.head()
import gc
del ml_df
gc.collect()

In [ ]:
df=df.merge(aboundance_df,left_on='Gene_id',right_on='Gene ID',how='left')

In [ ]:
KnownComponents={
'Tb11.v5.0765': 'TbPIGA',
'Tb927.5.3680': 'TbPIGH',
'Tb927.10.10110': 'TbPIGP',
'Tb927.10.6140': 'TbPIGC',
'Tb927.3.4570': 'TbPIGQ',
'Tb927.4.780': 'TbPIGY',
'Tb927.10.4610': 'DPM1',
'Tb927.9.6440': 'DPM2'}

In [ ]:
df['desc2'] = [ KnownComponents[g] if g in KnownComponents else d 
               for g,d in zip(df['Gene_id'],df['desc'])]

In [ ]:
cols = ['Gene_id','desc2','Peptide counts (razor+unique)','Score','Q-value','BSF_rank']
df[df['Gene_id'].isin(KnownComponents.keys())][cols]

In [ ]:
#list(df.columns)

In [ ]:
#color palette
palette=['b','b','b','r','r','r']
palette_g = ['r','b']
color_dictionary = { 'b':'Bait','r':'Control'}

In [ ]:
tag = 'iBAQ'
#tag='MS/MS count'
bait_cols = [tag+' Bait_'+str(n) for n in range(1,4)]
control_cols = [tag+' Control_'+str(n) for n in range(1,4)]
cols = bait_cols+control_cols
selection = df[cols]
selection.columns = [n.replace(tag+' ','') for n in selection.columns]
selection.head()

In [ ]:
print(selection.shape)
all_na = selection.replace(0,np.nan).isna().sum(axis=1)==6
selection=selection[~all_na]
print(selection.shape)

## iBAQ analysis 

In [ ]:
plt.style.use('ggplot')
fig,axes=plt.subplots(ncols=2,nrows=1,figsize=(14,4))

np.log10(selection.replace(0,np.nan)).plot(
    kind='kde', color=palette, alpha=0.5,ax=axes[0])
axes[0].legend(loc='center left', bbox_to_anchor=(0.7, 0.5))
axes[0].set_title('Value Distribution')
axes[0].set_xlabel('Log10 Intensity')

sns.boxplot(data =np.log10(selection.replace(0,np.nan)),
            showfliers=False,palette=palette,ax=axes[1])
plt.title('Value Distribution')
plt.xlabel('Log10 Intensity')
plt.savefig(os.path.join(TXT_PATH, 'value_distribution.png'))
plt.show()

In [ ]:
normed_df = selection.copy()
normed_df[['Bait_1','Bait_2','Bait_3']] = normed_df[['Bait_1','Bait_2','Bait_3']].replace(0,np.nan)
normed_df[['Bait_1','Bait_2','Bait_3']] = normed_df[['Bait_1','Bait_2','Bait_3']]/normed_df[['Bait_1','Bait_2','Bait_3']].median()

normed_df[['Control_1','Control_2','Control_3']] = normed_df[['Control_1','Control_2','Control_3']].replace(0,np.nan)
normed_df[['Control_1','Control_2','Control_3']] = normed_df[['Control_1','Control_2','Control_3']]/normed_df[['Control_1','Control_2','Control_3']].median()


fig,axes=plt.subplots(ncols=2,nrows=1,figsize=(14,4))

np.log10(normed_df.replace(0,np.nan)).plot(
    kind='kde', color=palette, alpha=0.5,ax=axes[0])
axes[0].legend(loc='center left', bbox_to_anchor=(0.7, 0.5))
axes[0].set_title('Value Distribution')
axes[0].set_xlabel('Log10 Intensity')

sns.boxplot(data =np.log10(normed_df.replace(0,np.nan)),
            showfliers=False,palette=palette,ax=axes[1])
plt.title('Value Distribution')
plt.xlabel('Log10 Intensity')
plt.savefig(os.path.join(TXT_PATH, 'value_distribution.png'))
plt.show()



## Missing Data

In [ ]:
import missingno as msno
import numpy as np
import matplotlib.pyplot as plt
#visualization of missing data
ax=msno.bar(selection.replace(0,np.nan),figsize=(6, 6))
plt.title('Missing Data Analysis',size=12)
ax.set_ylabel('Fraction of data points',size=12)
plt.tight_layout()
plt.savefig('missing.png')
plt.show()

## Proteins Found only in bait

In [ ]:
bait_nan = selection[['Bait_1','Bait_2','Bait_3']].replace(0,np.nan).isna().sum(axis=1)
control_nan  = selection[['Control_1','Control_2','Control_3']].replace(0,np.nan).isna().sum(axis=1)
only_bait = selection[(bait_nan==0) & (control_nan==3)]
only_control = selection[(bait_nan==3) & (control_nan<3)]
print(' Found only in Bait:',only_bait.shape[0],'\n',
      'Found only in Control:',only_control.shape[0])

In [ ]:
#selection[['Bait_1','Bait_2','Bait_3']] = selection[['Bait_1','Bait_2','Bait_3']].replace(0,np.nan)
#selection[['Bait_1','Bait_2','Bait_3']] = selection[['Bait_1','Bait_2','Bait_3']]/selection[['Bait_1','Bait_2','Bait_3']].median()
only_bait['mean']=normed_df[['Bait_1','Bait_2','Bait_3']].replace(0,np.nan).mean(axis=1)
to_look = only_bait.sort_values('mean', ascending=False).index.values
temp = df.loc[to_look]
temp=temp.join(normed_df[['Bait_1','Bait_2','Bait_3']])
temp['mean']=temp[['Bait_1','Bait_2','Bait_3']].replace(0,np.nan).mean(axis=1,skipna=True)
temp['exp_rank']=temp['mean'].rank(ascending=False)
temp['rank_filter']=np.log2(temp['BSF_rank']/temp['exp_rank'])
temp[['Gene_id','mean','Bait_1','Bait_2','Bait_3','desc2','Score','BSF_rank','exp_rank','rank_filter']].head(20)

In [ ]:
temp.to_csv('only_bait.csv')

In [ ]:
#print(selection.shape)
#selection=selection[(bait_nan==0)]
#print(selection.shape)

## Differential Abundance

### ProtRank indata

In [ ]:
indata_prank=selection.join(df[['Gene_id']],how='left')
indata_prank.set_index('Gene_id',drop=True,inplace=True)
indata_prank=indata_prank
indata_prank.to_csv(os.path.join(TXT_PATH, 'indata_ProtRank.csv'))
indata_prank.head()

In [ ]:
import ProtRank

### ProtRank Test

In [ ]:
what_to_compare = [[['Control_1', 'Bait_1'], ['Control_2', 'Bait_2' ], ['Control_3', 'Bait_3', ]]]
ProtRank.data_stats(indata_prank, what_to_compare = what_to_compare)
description = 'bait_vs_control'
significant_proteins = ProtRank.rank_proteins(
    indata_prank+indata_prank[indata_prank>0].min(), 
    what_to_compare, description,
    path_to=TXT_PATH,num_bootstrap_realizations=100)

In [ ]:
df[df['Gene_id'].isin(significant_proteins)][['Gene_id',
                                              'desc2']]

In [ ]:
temp_1 = pd.read_csv(os.path.join(TXT_PATH,'prs-bait_vs_control.dat'),
                   sep='\t', comment='#', index_col=[0],
                   names=['Gene_id','rank','FDR','sign'])
temp_1['log_FDR']=-np.log10(temp_1['FDR']+0.01)
temp_1['log_rank']=np.log10(temp_1['rank'])
temp_1['srank']=[n*1 if a=='+' else n*-1 for n,a in zip(temp_1['rank'],temp_1['sign'])]
temp_1=temp_1.merge(df,left_on='Gene_id',right_on='Gene_id',how='left')
#temp_1=temp_1.merge(aboundance_df,left_on='Gene_id',right_on='Gene ID',how='left')
temp_1['exp_rank']=temp_1['rank'].rank(ascending=False)
temp_1['rank_filter']=np.log2(temp_1['BSF_rank']/temp_1['exp_rank'])
temp_1['BSF_rank2']=temp_1['BSF_rank'].fillna(temp_1['BSF_rank'].max())

temp_1.head(50)
temp_1.to_csv('all.txt')

## Fig2 Static

In [ ]:
plt.style.use('fivethirtyeight')
#plt.style.use('classic')

fig, axes=plt.subplots(figsize=(16,8), 
                     ncols=2, nrows=1)

ids = temp_1[temp_1['srank']>40]
_ids = ids.index.values
_names = ids['desc2'].values
print(len(_ids))
print(len(_names))
ax=axes[0]
PTUT.make_vulcano(temp_1, ax, x='srank', y='log_FDR', 
             annot_index=_ids,
             annot_names = _names,
             title='',
             #fc_limit=70,
             fc_col='srank',
                  alpha_main=0.3,
             point_size_selection=30,
             point_size_all=10,
            text_size=12,          
            )

#ax.set_title('PIG-A interacting partners')
ax.set_xlabel('Experiment Rank')
ax.set_ylabel('Log10 FDR')
#ax.legend(loc='upper center', bbox_to_anchor=(1.1, 0.9),
#          title='Legend')
ax.yaxis.label.set_size(18)
ax.xaxis.label.set_size(18)
ax.title.set_size(18)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.text(-0.1, 1, 'A)', horizontalalignment='center', 
               verticalalignment='center',
               transform=ax.transAxes,fontsize=16)

ax=axes[1]
PTUT.make_vulcano(temp_1, ax, x='srank', 
                  y='BSF_rank2', 
             annot_index=_ids,
             annot_names = _names,
             title='',
             #fc_limit=70,
             fc_col='srank',
            alpha_main=0.3,
             point_size_selection=30,
             point_size_all=10,
            text_size=12,          
            )

#ax.set_title('PIG-A interacting partners')
ax.set_xlabel('Experiment Rank')
ax.set_ylabel('BSF Abundance Rank')
#ax.legend(loc='upper center', bbox_to_anchor=(1.1, 0.9),
#          title='Legend')
ax.yaxis.label.set_size(18)
ax.xaxis.label.set_size(18)
ax.title.set_size(18)
ax.tick_params(axis='both', which='major', labelsize=16)

ax.text(-0.1, 1, 'B)', horizontalalignment='center', 
               verticalalignment='center',
               transform=ax.transAxes,fontsize=16)

plt.suptitle('TbPIGA Interacting Partners')

plt.show()
plt.savefig('Fig2.svg')

## Fig2 Interactive

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots



fig1 = px.scatter(temp_1, x='srank', y='log_FDR', #color="log_FDR",
                 #title='PIGA interacting partners',
                 width=800, height=600,
                 hover_name="Gene_id", hover_data=["desc2"],
                 #color_continuous_scale='Bluered',#px.colors.sequential.Viridis,
                )


fig2 = px.scatter(temp_1, x='srank', y='BSF_rank2', #color="log_FDR",
                 #title='PIGA interacting partners',
                 width=800, height=600,
                 hover_name="Gene_id", hover_data=["desc2"],
                 #color_continuous_scale='Bluered',#px.colors.sequential.Viridis,
                )



trace1 = fig1['data'][0]
trace2 = fig2['data'][0]

fig = make_subplots(rows=1, cols=2, shared_xaxes=False,shared_yaxes=False)
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)

#fig.show()
fig.write_html("PIGA.html")

In [ ]:
from IPython.display import IFrame
IFrame("PIGA.html", width=1000, height=600)

In [ ]:
temp_1.to_csv('Dataset.csv')

In [ ]:
!jupyter nbconvert --to html_toc analysis_def.ipynb

In [5]:
!git add analysis_def*
!git add PIGA.html
!git add ProLib/ProteomicsUtility/utilities.py
!git add -u
!git commit -m 'new fig2'
!git push -u origin master

fatal: pathspec 'ProLib/ProteomicsUtility/utilities.py' is beyond a symbolic link
[master e79c9eb] new fig2
 Committer: Michele Tinti <mtinti@ls29490.local>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 1 file changed, 756 insertions(+), 2054 deletions(-)
 rewrite analysis_def.ipynb (94%)
Counting objects: 3, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 607 bytes | 0 bytes/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/mtinti/PIG-A.git
   5088dde..e79c9eb  master -> master
Branch